In [1]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [2]:
#change path
path="..\\UB_keystroke_dataset\\S0"

In [3]:
def ReadListOfTextFilesFromDirectoryWalk(path): 
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r, file))
    return files



In [4]:
files=ReadListOfTextFilesFromDirectoryWalk(path)
len(files)
print(files[0:2])

['..\\UB_keystroke_dataset\\S0\\baseline\\001000.txt', '..\\UB_keystroke_dataset\\S0\\baseline\\001001.txt']


In [44]:
def CreateListFromFile(file):
#Reading file and create tuple 
    items = []
#    iterator = (files,) if not isinstance(files, (tuple, list)) else files
#    for file in iterator:
    filename=file.split('\\')[-1].split('.')[0]
    f=open(file, "r")
    for line in f:
        words= line.rstrip('\n').split(sep=" ")
        items.append((words[0],words[1],words[2],filename))
    f.close()
    return items

In [47]:
items = []
for user in files[0:2]:
    items.append(CreateListFromFile(user))

In [7]:
#Creating dataframes
def CreateDataFrame(items,columns):
    df = pd.DataFrame.from_records(items, columns=['Key', 'EventType','Time','User'])
    return df

In [9]:
def ParseAlphabetsKeys(df):
    dfAlphabets=df[df["Key"].str.match('^.*[A-Z]$')]
    return dfAlphabets

In [48]:
def GetTimeTakenByKeys(data):
    keyDownsList = []
    rows=[]
    for row in data:
        if row[1]=="KeyDown":
            keyDownsList.append(row)
        elif row[1]=="KeyUp":
            i=0
            for keyDown in keyDownsList:
                if row[0]==keyDown[0]:
                    rows.append([ keyDown[0], int(row[2])- int(keyDown[2]),row[3]])
                    del keyDownsList[i]
                i = i + 1
    return rows

In [51]:
keyPressTime = GetTimeTakenByKeys(data=items[0])
keyPressTime

[['LMenu', 219, '001000'],
 ['Tab', 124, '001000'],
 ['LMenu', 187, '001000'],
 ['Tab', 140, '001000'],
 ['LMenu', 187, '001000'],
 ['Tab', 156, '001000'],
 ['Tab', 140, '001000'],
 ['Tab', 93, '001000'],
 ['LMenu', 1248, '001000'],
 ['D0', 78, '001000'],
 ['D0', 94, '001000'],
 ['D0', 94, '001000'],
 ['D5', 78, '001000'],
 ['Return', 62, '001000'],
 ['Return', 62, '001000'],
 ['Tab', 171, '001000'],
 ['LMenu', 905, '001000'],
 ['LMenu', 250, '001000'],
 ['Tab', 156, '001000'],
 ['Tab', 156, '001000'],
 ['LMenu', 296, '001000'],
 ['LMenu', 124, '001000'],
 ['Tab', 125, '001000'],
 ['I', 94, '001000'],
 ['LShiftKey', 280, '001000'],
 ['Space', 109, '001000'],
 ['A', 93, '001000'],
 ['M', 125, '001000'],
 ['Space', 125, '001000'],
 ['H', 125, '001000'],
 ['O', 125, '001000'],
 ['N', 125, '001000'],
 ['E', 109, '001000'],
 ['Back', 78, '001000'],
 ['O', 140, '001000'],
 ['R', 125, '001000'],
 ['E', 94, '001000'],
 ['D', 125, '001000'],
 ['Space', 93, '001000'],
 ['T', 109, '001000'],
 ['O

In [14]:
def GetTimeDifferenceofKeyDownDown(data):
    prev=0
    rows=[]
    for index,row in data.iterrows():
        if prev==0 and row[1]=="KeyDown":
            prevRow=row
            prev=1
        elif row[1]=="KeyDown":
            rows.append([ prevRow[0], row[0], int(row[2])- int(prevRow[2]),row[3]])
            prevRow=row
    return rows

In [33]:
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=temp)
keyDownDownTimeDifference

[['I', 'A', 359, '001000'],
 ['A', 'M', 296, '001000'],
 ['M', 'H', 359, '001000'],
 ['H', 'O', 140, '001000'],
 ['O', 'N', 141, '001000'],
 ['N', 'E', 296, '001000'],
 ['E', 'O', 2980, '001000'],
 ['O', 'R', 62, '001000'],
 ['R', 'E', 78, '001000'],
 ['E', 'D', 515, '001000'],
 ['D', 'T', 281, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'B', 250, '001000'],
 ['B', 'E', 187, '001000'],
 ['E', 'W', 343, '001000'],
 ['W', 'I', 140, '001000'],
 ['I', 'T', 63, '001000'],
 ['T', 'H', 1092, '001000'],
 ['H', 'Y', 1357, '001000'],
 ['Y', 'O', 172, '001000'],
 ['O', 'U', 62, '001000'],
 ['U', 'T', 203, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'D', 140, '001000'],
 ['D', 'A', 63, '001000'],
 ['A', 'Y', 343, '001000'],
 ['Y', 'A', 1076, '001000'],
 ['A', 'A', 983, '001000'],
 ['A', 'T', 141, '001000'],
 ['T', 'Y', 374, '001000'],
 ['Y', 'O', 187, '001000'],
 ['O', 'U', 78, '001000'],
 ['U', 'R', 141, '001000'],
 ['R', 'C', 405, '001000'],
 ['C', 'O', 109, '001000'],
 ['O', 'M', 172, '0010

In [40]:
featureVec = []
for i in range (len(keyDownDownTimeDifference)):
    featureVec.append([keyDownDownTimeDifference[i][3],keyDownDownTimeDifference[i][0],keyDownDownTimeDifference[i][1],keyPressTime[i][1],keyPressTime[i+1][1],keyDownDownTimeDifference[i][2]])

In [41]:
featureVec

[['001000', 'I', 'A', 94, 93, 359],
 ['001000', 'A', 'M', 93, 125, 296],
 ['001000', 'M', 'H', 125, 125, 359],
 ['001000', 'H', 'O', 125, 125, 140],
 ['001000', 'O', 'N', 125, 125, 141],
 ['001000', 'N', 'E', 125, 109, 296],
 ['001000', 'E', 'O', 109, 140, 2980],
 ['001000', 'O', 'R', 140, 125, 62],
 ['001000', 'R', 'E', 125, 94, 78],
 ['001000', 'E', 'D', 94, 125, 515],
 ['001000', 'D', 'T', 125, 109, 281],
 ['001000', 'T', 'O', 109, 109, 187],
 ['001000', 'O', 'B', 109, 93, 250],
 ['001000', 'B', 'E', 93, 140, 187],
 ['001000', 'E', 'W', 140, 94, 343],
 ['001000', 'W', 'I', 94, 47, 140],
 ['001000', 'I', 'T', 47, 109, 63],
 ['001000', 'T', 'H', 109, 62, 1092],
 ['001000', 'H', 'Y', 62, 125, 1357],
 ['001000', 'Y', 'O', 125, 140, 172],
 ['001000', 'O', 'U', 140, 140, 62],
 ['001000', 'U', 'T', 140, 109, 203],
 ['001000', 'T', 'O', 109, 156, 187],
 ['001000', 'O', 'D', 156, 94, 140],
 ['001000', 'D', 'A', 94, 109, 63],
 ['001000', 'A', 'Y', 109, 109, 343],
 ['001000', 'Y', 'A', 109, 12

In [16]:
def GetDictionaryOfFeatureVectors(keyDownDownTimeDifference):
    """
    Take List of arrays and return dictionary of 26*26 vector for each user
    """
    dictFV=dict()
    for xi in keyDownDownTimeDifference:
        if xi[3] not in dictFV:
            dictFV[xi[3]]=np.zeros(26*26,dtype=object)
            
        index=(ord(xi[0])-65)*26+np.absolute(ord(xi[1])-65)
        if dictFV[xi[3]][index]==0:
            dictFV[xi[3]][index]=[]
        dictFV[xi[3]][index].append(xi[2])
    return dictFV


In [17]:
FeatureVectors

NameError: name 'FeatureVectors' is not defined

In [18]:
def GetAverageFeatureVectors(FeatureVector):
    averageFV= np.zeros(26*26,dtype=np.float)
    index=0
    for xi in FeatureVector:
        tempSum=np.array(xi).sum()
        if type(xi)== list:
            averageFV[index]=tempSum/len(xi)
        index=index+1
    return averageFV

In [19]:
X = np.zeros(shape=(len(FeatureVectors),26*26),dtype=object)
index = 0
#X = np.array()
for user in FeatureVectors:
    X[index] = GetAverageFeatureVectors(FeatureVectors[user])
    index = index + 1

NameError: name 'FeatureVectors' is not defined

In [19]:
X.shape

(0, 676)

In [20]:
Y = np.zeros(len(FeatureVectors),dtype=int)
index = 0
for user in FeatureVectors:
    Y[index] = user
    index = index + 1

In [21]:
Y

array([], dtype=int32)

In [22]:
shape = (4, 6)

zeros_array = np.empty(26*26)

In [23]:
zeros_array

array([9.61363900e-312, 9.61351981e-312, 0.00000000e+000, 9.61363600e-312,
       4.94065646e-324, 9.61363600e-312, 4.94065646e-324, 9.61363600e-312,
       9.61264093e-312, 9.61383140e-312, 9.61264093e-312, 9.61264093e-312,
       1.48219694e-323, 1.38338381e-322, 9.61342520e-312, 9.61264093e-312,
       9.61264093e-312, 2.97079411e-313, 1.23516411e-322, 9.61363600e-312,
       9.61336114e-312, 1.90979621e-313, 2.97079411e-313, 4.94065646e-324,
       9.61363600e-312, 2.54639495e-313, 7.16045198e-315, 9.61363600e-312,
       9.61264093e-312, 4.88059032e-313, 1.90979621e-313, 7.16045198e-315,
       9.61363600e-312, 9.61363600e-312, 5.51718906e-313, 9.61264093e-312,
       9.61363600e-312, 1.48219694e-323, 9.61363600e-312, 4.88059032e-313,
       0.00000000e+000, 0.00000000e+000, 9.61264093e-312, 9.61363600e-312,
       9.61363600e-312, 9.61363600e-312, 2.97079411e-313, 9.61264093e-312,
       9.61363600e-312, 9.61363600e-312, 2.97079411e-313, 8.48798318e-314,
       9.61363600e-312, 1

In [29]:
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, Convolution1D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
import keras

In [30]:
total_sample_size = 5
num_of_pairs = 2
dim1 = 26*26
dim2 = 1
x_geuine_pair = np.zeros([total_sample_size, num_of_pairs, 1, dim1])
y_genuine = np.zeros([total_sample_size, 1])


x_imposite_pair = np.zeros([total_sample_size, num_of_pairs, 1, dim1])
y_imposite = np.zeros([total_sample_size, 1])

In [32]:
path1="..\\UB_keystroke_dataset\\S0"
sample1 = ReadListOfTextFilesFromDirectoryWalk(path1)
path2="..\\UB_keystroke_dataset\\S1"
sample2 = ReadListOfTextFilesFromDirectoryWalk(path2)

In [33]:
for count in range(total_sample_size):
    print(sample1[count])
    print(sample2[count])
    items = CreateListOfTuplesFromFile(sample1[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_geuine_pair[count, 0, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    items = CreateListOfTuplesFromFile(sample2[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_geuine_pair[count, 1, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    y_genuine[count] = 1


..\UB_keystroke_dataset\S0\baseline\001000.txt
..\UB_keystroke_dataset\S1\baseline\001100.txt
..\UB_keystroke_dataset\S0\baseline\001001.txt
..\UB_keystroke_dataset\S1\baseline\001101.txt
..\UB_keystroke_dataset\S0\baseline\002000.txt
..\UB_keystroke_dataset\S1\baseline\002100.txt
..\UB_keystroke_dataset\S0\baseline\002001.txt
..\UB_keystroke_dataset\S1\baseline\002101.txt
..\UB_keystroke_dataset\S0\baseline\003000.txt
..\UB_keystroke_dataset\S1\baseline\003100.txt


In [34]:
for count in range(total_sample_size):
    while True:
        ind1 = np.random.randint(total_sample_size)
        ind2 = np.random.randint(total_sample_size)
        if ind1 != ind2:
            break
            
    print(sample1[ind1])
    print(sample2[ind2])

    items = CreateListOfTuplesFromFile(sample1[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_imposite_pair[count, 0, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    items = CreateListOfTuplesFromFile(sample2[count])
    df = CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
    keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=ParseAlphabetsKeys(df))
    FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference)
    x_imposite_pair[count, 1, 0, :] = GetAverageFeatureVectors(FeatureVectors[list(FeatureVectors.keys())[0]])
    
    y_imposite[count] = 0


..\UB_keystroke_dataset\S0\baseline\003000.txt
..\UB_keystroke_dataset\S1\baseline\001100.txt
..\UB_keystroke_dataset\S0\baseline\002001.txt
..\UB_keystroke_dataset\S1\baseline\001100.txt
..\UB_keystroke_dataset\S0\baseline\001001.txt
..\UB_keystroke_dataset\S1\baseline\003100.txt
..\UB_keystroke_dataset\S0\baseline\001001.txt
..\UB_keystroke_dataset\S1\baseline\002100.txt
..\UB_keystroke_dataset\S0\baseline\001000.txt
..\UB_keystroke_dataset\S1\baseline\003100.txt


In [36]:
x_imposite_pair.shape

(5, 2, 1, 676)

In [37]:
X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
Y = np.concatenate([y_genuine, y_imposite], axis=0)

In [38]:
Y.shape
X.shape

(10, 2, 1, 676)

In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [44]:
def build_base_network(input_shape):
    
    #seq = Sequential()
    
    
    #nb_filter = [6, 12]
    #kernel_size = 3
    
    
    ##########################
    model_m = Sequential()
  #  model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
    #model_m.add(Conv1D(10, 1, activation='relu', input_shape=input_shape))
    #model_m.add(Conv1D(10, 1, activation='relu'))
 #   model_m.add(MaxPooling1D(3))
 #   model_m.add(Conv1D(160, 10, activation='relu'))
 #   model_m.add(Conv1D(160, 10, activation='relu'))
    #model_m.add(GlobalAveragePooling1D())
    #model_m.add(Flatten())
    #model_m.add(Dropout(0.5))
    model_m.add(Dense(1, activation='softmax', input_shape=input_shape))
    print(model_m.summary())
    ##########################

    return model_m

In [45]:
input_dim = x_train.shape[2:]
print(input_dim)
#img_a = Input(shape=input_dim)
#img_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
#feat_vecs_a = base_network(img_a)
#feat_vecs_b = base_network(img_b)

(1, 676)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1, 1)              677       
Total params: 677
Trainable params: 677
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

base_network.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 5
EPOCHS = 5



In [47]:
history = base_network.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

ValueError: Error when checking input: expected dense_3_input to have 3 dimensions, but got array with shape (7, 2, 1, 676)

In [81]:
input_dim

(1, 676, 1)

In [34]:
with open(sample1[count]) as file:
    data = file.read()

In [40]:
items = []
iterator = (sample1[count],) if not isinstance(sample1[count], (tuple, list)) else sample1[count]
for file in iterator:
    filename=file.split('\\')[-1].split('.')[0]
    f=open(file, "r")
    for line in f:
        words= line.rstrip('\n').split(sep=" ")
        items.append((words[0],words[1],words[2],filename))
    f.close()
